In [1]:
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

# The chess interface and game loop

### Dependencies
In the following all required dependencies, modules or notebooks are imported

In [2]:
%pip install python-chess

import chess
import chess.svg # TODO remove?
import chess.polyglot # TODO remove?
import chess.gaviota # TODO remove?
import time
import random
import json
from IPython.display import clear_output, SVG, display, HTML
from typing import Callable, Union

import import_ipynb
import Constants

#### chess.Board.push_legal
Pushes a move to a board's move stack only if it is a valid, legal move. If the move is illegal or invalid, a ValueError is raised and the board state remains unaffected.

###### <b><u>Arguments</u></b>
<b>move</b> <i>(chess.Move)</i> : The move that should be pushed to the move stack.

###### <b><u>Side effects</u></b>
Pushes a move to the board's move stack, assuming the move was valid and legal.

In [3]:
def push_legal(self, move: chess.Move):
    if move not in self.legal_moves:
        raise ValueError('Illegal move')
    self.push(move)

chess.Board.push_legal = push_legal
del push_legal

---
<br>

### Game class
Lets the user play a game of chess against the AI using a virtual chess board UI. The game may use any algorithm for the AI to make its move.
<br><br>

#### <u><i>Class variables</u></i>
<b>HTML_CONTENT</b> <i>String</i>        : The content which is displayed next to the chess board. <br> 
<b>HTML_HEADER</b> <i>String</i>         : The Header of the HTML-content. <br>
<b>BOARD_DISPLAY_SIZE</b> <i>Integer</i> : The size of the board in pixels. <br>

    ---

<b>board</b> <i>chess.Board</i>            : The chess board instance that's used for the game.<br>
<b>make_move</b> <i>dictionary(chess.COLOR : Callable)</i> : A dictionary of algorithms the AI uses to make a move according to the      
color-representation of the AI.<br>
<b>is_ai</b> <i>dictionary(chess.COLOR     : Boolean)</i> : A dictionary, where the chess.COLOR indicates, whether or not this color is controlled by
an AI. <br>
<b>move_times</b> <i>dictionary(chess.COLOR : List)</i> : A dictionary with Lists, inheriting the times an AI takes to move according to the 
chess.COLOR the AI possesses. <br>
<b>opening_book</b> <i>String</i> : A String, representing a path to an opening library. <br>
<b>is_first_move</b> <i>Boolean</i> : A boolean, representing whether or not the first move has already been executed
#### \_\_init\_\_
The constructor for the Game class. Gets called whenever a new Game instance is created.
###### __<u>Arguments</u>__
<b>make_move_algorithm_white</b> <i>Callable(chess.Board) [Optional] (Default behavior : None)</i>: The function that the AI controlling the white pieces should use to make a new move.<br>
__make_move_algorithm_black__ _Callable(chess.Board) [Optional] (Default behavior: None)_ : The function that the AI controlling the black pieces should use to make a new move. <br>
__opening_book__ _String [Optional] (Default behavior : None)_ : A String representing the path to the desired opening library the AIs will use. <br>
###### __<u>Side effects</u>__
If no AI is selected as at least one player, the game will exit. At least one AI is required to play the game.


In [4]:
class Game:
    # Constants
    HTML_CONTENT = 'html stuff'
    HTML_HEADER = 'Header'
    BOARD_DISPLAY_SIZE = 500

    # Board variables
    board = None

    # AI and player declarations
    make_move = None
    is_ai = None

    # Testing declarations
    move_times = None
    debug = None
    auto_tests = None
    wins = None

    # Opening book and endgame tablebase declarations
    opening_book = None
    endgame_tablebase_dir = None
    endgame_tablebase = None

    def __init__(
            self,
            make_move_algorithm_white: Callable=None,
            make_move_algorithm_black: Callable=None,
            opening_book: str='Resources/baron30.bin',
            endgame_tablebase_dir: str='/datasets/gaviota'
    ):
        self.opening_book = opening_book
        self.endgame_tablebase_dir = endgame_tablebase_dir

        self.debug = {}
        self.auto_tests = {}
        self.wins = []

        self.board = chess.Board()
        self.board.ending = False

        self.move_times = { chess.WHITE: [], chess.BLACK: [] }
        self.make_move = { chess.WHITE: make_move_algorithm_white, chess.BLACK: make_move_algorithm_black }
        self.is_ai = self.get_ai_state()

        if self.endgame_tablebase_dir is not None:
            self.endgame_tablebase = chess.gaviota.open_tablebase(self.endgame_tablebase_dir)

        # The game requires that at least one player is an AI - it is not a multiplayer chess game
        if not self.is_ai[chess.WHITE] and not self.is_ai[chess.BLACK]:
            raise ValueError('At least one player must be an AI')


### play
Plays the game.
Depending on the input arguments there are different game-states and features:
1. (AI, AI): 
    - Displays the chess board visually.
    - Lets the AI make a move based on the board state and specified algorithm.
    - Lets the other AI make a move based on the board state and specified algorithm.
2. (AI, Player) - (Player, AI):
    - Displays the chess board visually.
    - Lets the AI make a move based on the board state.
    - Lets the human player input their move and pushes it to the board 
    (if it's valid and legal).
3. (Player, Player): 
    - Game will exit -> At least one AI is required to play the game

######  <u><i>Side effects</i></u> 
- The state of the board in the Game class is changed.  
- The state of is_first_move is changed after the first move.
- The times the AIs take to make a move is tracked and saved.
- The chess board is visually displayed.


In [5]:
def play(self) -> None:
    self.debug.clear() # clear the debug dictionary - this dict may have contained moves from the last playthrough
    while not self.board.is_game_over():
        self.display()
        if self.is_ai[self.board.turn]:
            # Start time
            time_stamp = time.time()
            if not self.make_opening_move():
                score, move = self.make_move[self.board.turn](self.board, self.board.turn, self.endgame_tablebase)
                self.debug[len(self.board.move_stack)] = (score, move.uci())
            # Find time difference
            time_stamp = time.time() - time_stamp
            # Add time to move_times
            self.move_times[self.board.turn].append(time_stamp)
        else:
            self.make_move_human()

        if self.endgame_tablebase is not None:
            try:
                self.HTML_CONTENT = ''
                self.HTML_CONTENT += 'DTM:\n'
                self.HTML_CONTENT += str(self.endgame_tablebase.probe_dtm(self.board)) + '\n'
                self.HTML_CONTENT += 'WDL:\n'
                self.HTML_CONTENT += str(self.endgame_tablebase.probe_wdl(self.board)) + '\n'
            except KeyError:
                pass
        else:
            self.HTML_CONTENT = 'wut'

        self.board.check_and_set_ending()
        self.display()

    winner_message = 'It\'s a draw!' if self.board.outcome() is None or self.board.outcome().winner is None \
        else f'{"White" if self.board.outcome().winner == chess.WHITE else "Black"} is the winner!'
    print(f'The game has ended. {winner_message}\n')
    print(self.board.outcome()) 

Game.play = play
del play

### display
Displays the chess board graphically. Furthermore, custom HTML is displayed next to the board.

###### <u><i>Side effects</i></u>
The current visual output is overwritten.


In [6]:
def display(self) -> None:
    clear_output(wait=True)
    try:
        last_move = self.board.peek()
        board_visual = chess.svg.board(self.board, size=self.BOARD_DISPLAY_SIZE, lastmove=last_move, check=self.get_check_square()) 
    except IndexError:
        board_visual = chess.svg.board(self.board, size=self.BOARD_DISPLAY_SIZE)
    table = '<table><th>Board</th><th>{}</th><tr><td><div style=\"vertical-align: top; text-align: left\">{}</div></td><td>{}</td></div></tr></table>'
    display(HTML(table.format(self.HTML_HEADER, board_visual, self.HTML_CONTENT)))

Game.display = display
del display

### get_check_squares
checks for piecs or squares giving check.
###### <u><i>Returns</u></i>
__chess.Square__ _None [if no square sets gives check]_
###### <u><i>Side effects</u></i>
If get_check_squares returns a value, the display function will indicate the square giving check with a red background
    

In [7]:
def get_check_square(self) -> chess.Square:
    if self.board.is_check():
        return self.board.checkers().pop()
    else:
        return None
            
Game.get_check_square = get_check_square
del get_check_square

### make_opening_move
Makes a random move from the opening book, if available. If not, no move is made.
###### <u><i>Returns</u></i>
__True:__ A move from the opening book was made.  
__False:__ A move from the opening book was not made.


In [8]:
def make_opening_move(self) -> bool:
    if self.opening_book is None:
        return False

    with chess.polyglot.open_reader(self.opening_book) as reader:
        possible_moves = [ entry.move for entry in reader.find_all(self.board) ]
        if len(possible_moves) > 0:
            move = random.choice(possible_moves)
            self.board.push(move)
            self.debug[len(self.board.move_stack)] = ("opening library", move.uci())
            return True

    return False

Game.make_opening_move = make_opening_move
del make_opening_move

### get_ai_state
Define the AIs for the game. Dependent on the ai_algorithms given as arguments in game.play()
this functions defines the player state for the different colors. (AI or Player). If the 
"player" is defined as an ai, the dictionary value to the coresponding chess.Color is defined 
as True.
###### <u><i>Returns</u></i>
__{chess.Color: Bool}__ _Dict_


In [9]:
def get_ai_state(self) -> dict:
    return {
        color: self.make_move[color] is not None
        for color in ( chess.WHITE, chess.BLACK )
    }

Game.get_ai_state = get_ai_state
del get_ai_state


### make_move_human
If the "player" is human, this method enables the human user to make a move through the 
input-console.
###### <u><i>Side effects</u></i>
A move is pushed onto the move_stack and it is the next players turn.

In [10]:
def make_move_human(self) -> None:
    input_prompt = 'Please input your move: '
    while True:
        try:
            move = chess.Move.from_uci(input(input_prompt))
            self.board.push_legal(move)
            return
        except ValueError:
            input_prompt = 'Illegal move, please try again: '
                
Game.make_move_human = make_move_human
del make_move_human

### save_state
The movestack as well as all the scores are saved in a file, named by the user. The file is stored in the current working-directory. 
This file can then later be loaded to open a game.
       

In [11]:
def save_state(self) -> None:
    filename = input("Please input save-name:\n ") + ".txt"
    f = open(filename, 'w')
    state = json.dumps(self.debug)
    f.write(state)
    f.close()
    
Game.save_state = save_state
del save_state

### load_state
A saved game is loaded. Therefore, the game will be recreated by loading the move_stack from a saved game.
       
       

In [12]:
def load_game(self):
    try:
        f = open(input("What is your save-file called? :\n"))
    except OSError:
        print(f'There is no such file called {f} - please try again')
    s = f.read()
    d = json.loads(s)
    self.debug = d
    self.board.clear_stack()
    
    for _, moves in self.debug.values():
        self.board.push_legal(chess.Move.from_uci(moves))
    
Game.load_game = load_game
del load_game

______________________________________________________________

# Debug Section
<br>

**In this section, the Game.play() and Game.display() - Methods will be modified and saved as Game.debug_play() and Game.display_debug()**

The reason is, that a mode for debuging purpose is generally needed if we want to figure out the reason behind bugs or problems. 
Instead of using the tricky python debuger, we developed a method to show only the desired attributes when we need to see them. 
Through this, we can take a deeper look into the Game.play() method while changing the display()-method, so we can see through step by step or move by move.


### debug_play
This is the debugger-method for bug-detection. Here, a dictionary of chess-problems is given. The user can choose to present a certain problem as
argument of the method or choose the problem out of given problems while running the "debugger". Furthermore, it is possible to automate the procedure,
if the methods gets executed in a loop. (Example in Test.ipynb) Through this it is possible to run all tests and track all necessary information. If 
automation is not enabled, the information is not saved, due to the user beeing able to access it through the game-object. (If the method is called
twice or more, the information will be overwritten, so be careful to enable automation if you run multiple tests!).
Besides small differences to enable better debugging-opportunities, this function performs like Game.play(). 
###### <u><i>Arguments</u></i>
__chess_problem__ _Callable(String) [Optional] (Default behavior: None)_  
A string associated with a key of the Constants.chess_problems dictionary.  
__automation__ _Callable(Boolean) [Optional] (Default behavior: False)_  
A boolean indicating whether or not this call is part of a automatic test-run consisting of multiple tests.  
###### <u><i>Side effects</u></i>
__automation = False__  
If automation is set to false, the user has to press a button after every move. Therefore we can deside how long we want wait and look at the 
displayed details before the next move is executed.  
__automation = True__  
All tests run without interruption. After all tests ran, details can be accessed through game.auto_tests, a collection of all debug-dictionaries and 
game.wins, a collection of all winners ('white', 'black' or None -> Draw).    
  
The user has to press any key after each move.

In [13]:
def debug_play(self, chess_problem = None, automation = False) -> None:
    problem_dict = Constants.chess_problems
    self.debug.clear() # clear the debug dictionary - this dict may have contained moves from the last playthrough
    if chess_problem is None or chess_problem not in problem_dict:
        problem = input("Please choose chess-problem: \nAJW, \nAF, \nLE, \nMPN, \nEK, \nGBE \nyou can find further information about the given problems in Constants.ipynb\n")
        while problem not in problem_dict:
            clear_output()
            print("\nAJW, \nAF, \nLE, \nMPN, \nEK, \nGBE")
            problem = input("Wrong input, please try again. Type one of the above numbers to initialze the test")
    else: problem = chess_problem
    self.board.set_fen(problem_dict[problem])
    
    while not self.board.is_game_over():
        self.display_debug(automation)
        if self.is_ai[self.board.turn]:
            # Start time
            time_stamp = time.time()
            if not self.make_opening_move():
                score, move = self.make_move[self.board.turn](self.board, self.board.turn, self.endgame_tablebase)
                self.debug[len(self.board.move_stack)] = (score, move.uci())
            # Find time difference
            time_stamp = time.time() - time_stamp
            # Add time to move_times
            self.move_times[self.board.turn].append(time_stamp)
        else:
            self.make_move_human()

        if self.endgame_tablebase is not None:
            try:
                self.HTML_CONTENT = ''
                self.HTML_CONTENT += 'DTM:\n'
                self.HTML_CONTENT += str(self.endgame_tablebase.probe_dtm(self.board)) + '\n'
                self.HTML_CONTENT += 'WDL:\n'
                self.HTML_CONTENT += str(self.endgame_tablebase.probe_wdl(self.board)) + '\n'
            except KeyError:
                pass
        else:
            self.HTML_CONTENT = 'wut'

        self.board.check_and_set_ending()
        
    self.display_debug(True)
    winner_message = 'It\'s a draw!' if self.board.outcome() is None or self.board.outcome().winner is None \
        else f'{"White" if self.board.outcome().winner == chess.WHITE else "Black"} is the winner!'
    print(f'The game has ended. {winner_message}\n')
    print(self.board.outcome())
    if automation:
        self.auto_tests[problem] = self.debug
        winner = None
        if self.board.outcome() is None or self.board.outcome().winner is None:
            winner = None 
        elif self.board.outcome().winner == chess.WHITE:
            winner = "white"
        else: winner = "black"
            
        self.wins.append(winner)

Game.debug_play = debug_play
del debug_play

### display_debug
This is the display-method used in the debug_play-method. Besides the method waiting for the user to press a button, this method doesn't differ from the display-method.
    
###### <u><i>Arguments</u></i>
__Bool__ _Boolean_  
A boolean to determine if this call is one of multiple-automated tests.

###### <u><i>Side effects</u></i>
__Bool = True__  
The user has to press any key after each move.

In [14]:
def display_debug(self, Bool) -> None:
    clear_output()
    try:
        last_move = self.board.peek()
        board_visual = chess.svg.board(self.board, size=self.BOARD_DISPLAY_SIZE, lastmove=last_move, check=self.get_check_square()) 
    except IndexError:
        board_visual = chess.svg.board(self.board, size=self.BOARD_DISPLAY_SIZE)
    table = '<table><th>Board</th><th>{}</th><tr><td><div style=\"vertical-align: top; text-align: left\">{}</div></td><td>{}</td></div></tr></table>'
    display(HTML(table.format(self.HTML_HEADER, board_visual, self.HTML_CONTENT)))
    if(not Bool):
        if len(self.board.move_stack) != 0:
            print(self.debug[len(self.board.move_stack)])
        input("press any button to continue")

Game.display_debug = display_debug
del display_debug

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d6ce9acd-52c5-4422-904d-8424da19408b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>